In [ ]:
import cv2
import os
import numpy as np
import time
from IPython.display import display, clear_output, Image
import io

# Load labels, model config, weights
labelsPath = 'obj.names'
LABELS = open(labelsPath).read().strip().split("\n")
weightsPath = 'crop_weed_detection.weights'
configPath = 'crop_weed.cfg'

COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

# YOLO output layers
ln = net.getLayerNames()
layer_indexes = net.getUnconnectedOutLayers().flatten()
ln = [ln[i - 1] for i in layer_indexes]

# Confidence and NMS thresholds
confi = 0.5
thresh = 0.5

# Load video (or use 0 for webcam)
cap = cv2.VideoCapture("crop_weed_video.mp4")

frame_count = 0
max_frames = 100  # Set to your preferred limit

while True:
    ret, image = cap.read()
    if not ret or frame_count >= max_frames:
        break

    (H, W) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (512, 512), swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(ln)

    boxes = []
    confidences = []
    classIDs = []

    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence > confi:
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    idxs = cv2.dnn.NMSBoxes(boxes, confidences, confi, thresh)

    if len(idxs) > 0:
        for i in idxs.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            color = [int(c) for c in COLORS[classIDs[i]]]
            cv2.rectangle(image, (x, y), (x + w - 100, y + h - 100), color, 2)
            text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
            cv2.putText(image, text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    # Convert frame to JPEG for display
    _, buffer = cv2.imencode('.jpg', image)
    display(Image(data=buffer.tobytes()))
    clear_output(wait=True)

    frame_count += 1
    time.sleep(0.05)  # slight delay to mimic playback speed (~20 FPS)
    
cap.release()
print("[STATUS]   : Completed")
